In [ ]:
import tensorflow as tf 
from tensorflow import keras
import tensorflow_addons as tfa
import tensorflow_hub as hub 
from sklearn.metrics import mean_absolute_error 
import numpy as np 
import datetime

### Load data

In [2]:
train_ds = tf.data.experimental.load('./data/fullscene/train_ds') \
    .cache().shuffle(buffer_size=1000, seed=42).prefetch(buffer_size=tf.data.AUTOTUNE)

valid_ds = tf.data.experimental.load('./data/fullscene/val_ds') \
    .cache().shuffle(buffer_size=1000, seed=42).prefetch(buffer_size=tf.data.AUTOTUNE)

train_ds, valid_ds

Instructions for updating:
Use `tf.data.Dataset.load(...)` instead.


(<PrefetchDataset element_spec=(TensorSpec(shape=(None, 10, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 5), dtype=tf.float32, name=None))>,
 <PrefetchDataset element_spec=(TensorSpec(shape=(None, 10, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 5), dtype=tf.float32, name=None))>)

### Load VGG16

In [3]:
vgg16 = keras.applications.VGG16(include_top=False, weights='imagenet', pooling='max')
vgg16.trainable = False

### Load Vit-B16

In [4]:
vit_extractor = hub.KerasLayer("https://www.kaggle.com/models/spsayakpaul/vision-transformer/frameworks/TensorFlow2/variations/vit-b16-fe/versions/1/", trainable=False) # ../../vit_b16_fe/


ValueError: Trying to load a model of incompatible/unknown type. 'C:\Users\handi\AppData\Local\Temp\tfhub_modules\942538f3cac9601ab854e41314777943a13e7d25' contains neither 'saved_model.pb' nor 'saved_model.pbtxt'.

## Build scene model

In [ ]:
inputs = keras.layers.Input(shape=(10,224,224,3), name='Input')

# First model 
x      = keras.layers.TimeDistributed(keras.layers.Rescaling(scale=1./255.0), name='Rescaling')(inputs)
x      = keras.layers.TimeDistributed(vgg16, name='vgg16')(x)
x      = keras.layers.LSTM(units=128, return_sequences=True)(x)
x      = keras.layers.LSTM(units=64)(x)
x      = keras.layers.Dropout(0.2)(x)

x      = keras.layers.Dense(units=1024)(x)
x      = keras.layers.Dense(units=512, activation='relu')(x)


# Second model 
y      = keras.layers.TimeDistributed(keras.layers.Rescaling(scale=1./127.5, offset=-1))(inputs)

#vit_extractor = hub.KerasLayer("https://www.kaggle.com/models/spsayakpaul/vision-transformer/frameworks/TensorFlow2/variations/vit-b16-fe/versions/1/", trainable=False) # ../../vit_b16_fe/
vit_extractor.compute_output_shape = lambda x: (x[0], 768)

y      = keras.layers.TimeDistributed(vit_extractor)(y)
y      = keras.layers.LSTM(units=128, return_sequences=True)(y)
y      = keras.layers.LSTM(units=64)(y)
y      = keras.layers.Dropout(0.2)(y)
y      = keras.layers.Dense(units=1024)(y)
y      = keras.layers.Dense(units=512, activation='relu')(y)


# Averaget two models
z      = keras.layers.Average()([x,y])

z      = keras.layers.Dense(256, activation='relu')(z)
z      = keras.layers.Dropout(0.5)(z)

z      = keras.layers.Dense(5, activation='sigmoid')(z)

model  = keras.models.Model(inputs=inputs, outputs=z)
keras.utils.plot_model(model, show_shapes=True)

### Compile model

In [ ]:
t = datetime.datetime.now().strftime("%m%d_%H%M%S")

optimizer      = tfa.optimizers.RectifiedAdam()

early_stopping = keras.callbacks.EarlyStopping(patience=10, verbose=0)
check_point    = keras.callbacks.ModelCheckpoint(filepath='./weights/scene/scene.t5',
                             monitor='val_mae',
                             mode='min',
                             save_best_only=True,
                             save_weights_only=True,
                             verbose=0)

model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])

### Train

In [ ]:
history = model.fit(train_ds, validation_data=valid_ds, batch_size=2, epochs=100, callbacks=[early_stopping, check_point])

## Train Evaluation

### Load weights

In [ ]:
model.load_weights('./weights/scene/scene.t5')


## Evaluation

### Validation data

In [ ]:
valid_ds = tf.data.experimental.load('./data/fullscene/val_ds')
valid_ds

loss, mae = model.evaluate(valid_ds)
(1-mae)*100

In [ ]:
y_true = np.concatenate([y for x,y in valid_ds])
y_pred = model.predict(valid_ds)

mae = mean_absolute_error(y_true, y_pred, multioutput='raw_values')
(1-mae)*100, (1-np.mean(mae))*100

### Test data

In [ ]:
test_ds = tf.data.experimental.load('./data/fullscene/test_ds/')
loss, mae = model.evaluate(test_ds)
(1-mae)*100

In [ ]:
y_true = np.concatenate([y for x,y in test_ds], axis=0)
y_pred = model.predict(test_ds)

mae = mean_absolute_error(y_true, y_pred, multioutput='raw_values')
(1-mae)*100, (1-np.mean(mae)) * 100